# Building a d-layer Neural Network from Scratch to Classify MNIST
### by Eduardo Coronado (12/15/19)

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.metrics import accuracy_score, classification_report


Here we will construct a $d$-dimensional neural network with the following architecture 784 − $H$ −
$H$ - $\ldots$ − 1, where $H$ is the number of hidden nodes (neurons) per layer $d$ is the number of layers. We will use this to classify the number from the MNIST data set and show we can achive $>97\%$ accuracy. 

## Data

### Load and Extract

First, we load the pre-processed MNIST data found in this repo `mnist_all.mat` using the `sio` package. Where the pre-processing involved randomly splitting the original dataset into train and test sets and turning each image $\mathbf{x}_i$ into a $1x 784$ vector (i.e. $f : \mathbb{R}^{28x28} \rightarrow \mathbb{R}^{784}$).  

We then extract the train and test data per digit and store them into numpy arrays. The features `X_train` and `X_test` were normalized such that each value is between 0 and 1. Since MNIST images are black and white we can achive this by dividing $1\;/ \;255$, given $255$ is the max value in this type of images.

In [ ]:
# Load MNIST data
mnist = sio.loadmat("./mnist_all.mat")

# Extract pixel data/observations
X_test = [val for key, val in mnist.items() if "test" in key]
X_train = [val for key, val in mnist.items() if "train" in key]

# Normalize (1/255) given MNIST data is B&W images
X_test = np.vstack(X_test) / 255.
X_train = np.vstack(X_train)/ 255.
X_train, X_test = X_train.T, X_test.T

# Extract observation train and test labels observations
y_test = [np.full((1,val.shape[0]),key.replace("test",""), dtype=int ) 
          for key,val in mnist.items() if "test" in key]
y_train = [np.full((1,val.shape[0]),key.replace("train",""), dtype=int ) 
          for key,val in mnist.items() if "train" in key]

y_test = np.hstack(y_test)
y_train = np.hstack(y_train)

### One-hot encode labels
Since the MNIST data set contains 10 digits (i.e. 10 different labels), it is beneficial to one-hot encode the labels such that we obtain a $10xn$ and $10xm$ arrays for $n$ train and $m$ test labels. Here, each column represents the label for a single image and is sparse. For example, let $\mathbf{x}_j = \{x_{1j}, x_{2j}, \ldots, x_{nj}\}$ be the one-hot encoded label for observation $j$ and $x_{\cdot j} \in \{0,1\}$ be the entries of this column. So, if this observation's was labeled as digit 0 then $x_{1j} = 1$ and $x_{i,j} = 0  \quad \text{for }\; i= 2,\ldots,10$

Thus  $\mathbf{x}_j = \{1, 0, \ldots, 0\}$

In [ ]:
# One-hot encode train and test labels 10 x n_samples with 0,1 entries
y_test_new = np.eye(10)[y_test.astype('int32')]
y_test_new = y_test_new.T.reshape(10, y_test.shape[1])

y_train_new = np.eye(10)[y_train.astype('int32')]
y_train_new = y_train_new.T.reshape(10, y_train.shape[1])

## Define the Neural Network
Before we dive into any further coding, we will go over some basics.

First, let's define the networks high-level architecture such as the layer weights $w_j$, layer outputs $h_j$,  activations functions $\sigma$, and more.

We can represent the neural network as the following function 

$$f(x \mid \theta) : \mathbb{R}^{784} \rightarrow \{0,1\}$$, 

where the parameters $\theta$ of a network with $d$ layers is defined by

$$\theta = \{ \mathbf{W}_1 \in \mathbb{R}^{784 x H}, \mathbf{W}_j \in \mathbb{R}^{H x H}, \mathbf{w}_d \in \mathbb{R}^H, \mathbf{b}_k \in \mathbb{R}\}$$

where $k = 1, \ldots, d$ and $ 1< j <d$.

More explicitly, $f(x)$ is defined by the following
\begin{align*}
f(x) &= \sigma_m \left(\mathbf{w}_d^T h_{d-1} + \mathbf{b}_d \right)\\[1ex]
h_{d-1} &= \sigma \left(\mathbf{W}_{d-1}^T h_{d-2} + \mathbf{b}_{d-1} \right)\\[1ex]
&\vdots\\[1ex]
h_2 &= \sigma \left(\mathbf{W}_2^T h_1 + \mathbf{b}_2 \right)\\[1ex]
h_1 &= \sigma \left(\mathbf{W}_1^T \mathbf{x} + \mathbf{b}_1\right)
\end{align*}

where $\mathbf{x}$ is the input data, $b_i$ are the bias per layer, 

\begin{align*}
\sigma(\mathbf{z}) &= \frac{1}{1 + e^{-z}} \quad \text{(is a } \mathbf{sigmoid \; activation} \text{ function)}\\[1ex] \sigma_m(\mathbf{z})_i &= \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}} \quad \text{(is a } \mathbf{softmax \; activation} \text{ function to classify MNIST 10 digits (i.e. $K = 10$, 0 through 9 digits))}
\end{align*}

Similarly, we will use a **cross-entropy** loss function 

$$\mathcal{L}(\theta) = - \sum_{i=1}^N (y_i log f(\mathbf{x}_i) + (1 - y_i)log(1 - f(\mathbf{x}_i))$$

Having define the above we can now define some helper functions for the loss and activation functions

In [17]:
# Define some helper functions

# Sigmoid activation
def sigmoid(z):
    out = 1. / (1. + np.exp(-z))
    return out

# Softmax output activation for final layer(I decided to use this given build a multi-class NN)
def softmax(z):
    out_soft = np.exp(z)/ np.sum(np.exp(z), axis = 0)
    return out_soft

# Loss function: cross entropy
def cross_entropy(y_true, y_hat):
    loss_sum = np.sum(np.multiply(np.log(y_hat),y_true) ) + np.sum( np.multiply(np.log(1-y_hat),(1-y_true)))
    loss = - (1./y_true.shape[1]) * loss_sum

    return loss


## Main steps to train a Neural Network

Training a Neural Network involves three main steps,

1) **Initiate Parameters**: Randomly initiate weights $\mathbf{W}$ and biases $\mathbf{b}$

2) **Feedforward**: Compute the errors based on the predicted label assigments for each observation 

3) **Backpropagation**: Compute weight gradients to adjust weights based on error and loss function

**Repeat steps 2-3 above iteratively to generate a learning process**

### 1) Parameter Initialization

The first step before training a neural network is to initialize the weight and bias parameters for the number of layers and neurons/layer in the network. In this example, I've built a function that will initialize a neural net with $d$ layers (user defined) each with a default 64 neurons/layer and will output a dictionary `param` 
that will contain the weight $\mathbf{W}$ and bias $\mathbf{b}$ parameter values.

A common way to do so is randomly initializing $\mathbf{W} \sim N(0, \sigma^2)$ and $\mathbf{b} = \mathbf{0}$. However, it is important to choose a value of $\sigma^2$ such that the weights are similar in distribution for all layers and **avoid problems with vanishing or exploding gradients** later on. In other words, we'd like to achieve

$$Var(\mathbf{z}_l) = Var(\mathbf{z}_{l-1})$$

where $\mathbf{z}_l = \mathbf{W}_l^T \mathbf{h}_{l-1} + \mathbf{b}_l$.

After some simplification from first principles and taking into account that at each layer the weights have a symmetric, i.i.d. normal distribution we obtain the following optimal value for variance $\sigma^2$

$$\sigma^2_l = \frac{2}{n_l}$$


Therefore, during initialization we should make sure that the variance for each weight is $\frac{2}{n_l}$ which can be obtained by multiplying the $\mathbf{W}_l \sim N(0,1)$ random variable by $\sqrt{\frac{2}{n_l}}$ to obtain the desired variance. This is based on the quadratic properties of the variance of an i.i.d. random variable - $Var(aX) = a^2Var(X)$.
<br><br>

In this example, the $n_1 = 784$ and each layer after $n_2 = \ldots = n_d = 64$ since we are using vectors.

In [ ]:
# Initialize layer parameters
# Choose n_h = number of hidden nodes, 
# d = number of layers, 
# n_x = number of inputs per observation (i.e. 784 for MNIST)
def initialize_params(layers, n_h = 64, n_x = 784):
    np.random.seed(10)
    param = {}
    
    for i in range(1, layers + 1):
        w_str, b_str = "w" + str(i), "b" + str(i)
        
        # First layer
        if i == 1:
            # Use weight normalization after random sample from std normal based on Var derivation in part b
            param[w_str] = np.random.randn(n_h, n_x) * np.sqrt((2./ n_x)) 
            
            param[b_str] = np.zeros((n_h, 1)) * np.sqrt(2. / n_x)
            
        # Last layer
        elif i == d:
            param[w_str] = np.random.randn(10, n_h) * np.sqrt((2./ n_h))
            param[b_str] = np.zeros((10, 1)) * np.sqrt(2. / n_x)
        
        # Any intermediate layer
        else:
            param[w_str] = np.random.randn(n_h, n_h) * np.sqrt((2./ n_h))
            param[b_str] = np.zeros((n_h, 1)) * np.sqrt(2. / n_h)
            
    return param


### 2) Feedforward

Next, lets define the feedforward step based on the layout above. Here we compute the following to obtain the value of $f(\mathbf{x})$ for a single input $\mathbf{x}$.

\begin{align*}
h_1 &= \sigma \left(\mathbf{W}_1^T \mathbf{x} + \mathbf{b}_1\right)\\[1ex]
h_2 &= \sigma \left(\mathbf{W}_2^T h_1 + \mathbf{b}_2 \right)\\[1ex]
&\vdots\\[1ex]
h_{d-1} &= \sigma \left(\mathbf{W}_{d-1}^T h_{d-2} + \mathbf{b}_{d-1} \right)\\[1ex]
f(x) &= \sigma_m \left(\mathbf{w}_d^T h_{d-1} + \mathbf{b}_d \right)\\[1ex]
\end{align*}

where $\sigma$ is the sigmoid function and $\sigma_m$ is the softmax function.

In this example, we store this values in `dictionaries` to make the values easy to access. Let $\mathbf{z}$ be the inputs into the activation function $\sigma(\mathbf{z})$ coming from the previous layer, and $h$ be the output after the activation function in the current layer (i.e. $\mathbf{h} = \sigma(\mathbf{z})$).  

Here we define `fwd_dic` which the dictionary that stores all the layer inputs $\mathbf{z}_j$ and outputs $\mathbf{h}_j$, and leverage the `param` dictionary we used during initialization.


In [20]:
# Feedforward function
# Inputs: initialized param dictionary, X matrix (784 x n), and d layers
# Output: dictionary with forward computations for z_j and h_j activations
def forward(param, X, d):
    fwd_dic = {} # Initialize dictionary
    
    for i in range(1, d+1):
        z_str = "z" + str(i) # create z_j for activation functions h_j
        h_str = "h" + str(i) # activation z_j
        w_str, b_str = "w" + str(i), "b" + str(i) 
        
        # First layer computations z_1 and h_1
        if i == 1:
            fwd_dic[z_str] = np.matmul(param[w_str], X) + param[b_str]
            fwd_dic[h_str] = sigmoid(fwd_dic[z_str])
            
        # Final layer computations z_d and h_d
        elif i == d:
            # Previous layer's activation
            h_str_prev = "h" + str(i -1)
            prev_h = fwd_dic[h_str_prev]
            
            # Current layer activation
            fwd_dic[z_str] = np.matmul(param[w_str], prev_h) + param[b_str]
            fwd_dic[h_str] = softmax(fwd_dic[z_str]) # Softmax activation for multi-class
        
        # Intermediate layer computations
        else:
            # Previous layer's activation
            h_str_prev = "h" + str(i -1)
            prev_h = fwd_dic[h_str_prev]
            
            # Current layer activation
            fwd_dic[z_str] = np.matmul(param[w_str], prev_h) + param[b_str]
            fwd_dic[h_str] = sigmoid(fwd_dic[z_str])
      
    
    return fwd_dic





### 3) Backpropagation

After we compute the backpropagation step. Here we want to learn know how $\mathcal{L}$ changes with respect to each weight $\mathbf{W}_j$ and bias $\mathbf{b}_k$ for $j,k = 1, \ldots, d$. 

The learning process is normally done via gradient descent, however since the MNIST data set is large we can approximate the true gradient using **Stochastic Gradient Descent** which can be computed 

$$\nabla \mathcal{L} = \frac{1}{B} \sum_{b =1}^B \nabla \mathcal{L}_b$$

where at each step we randomly pick a mini-batch of $B$ samples and calculate the gradient based on this mini-batch.

#### Derive Gradients  and Stochastic Gradient Descent

Since we're interested in how the loss changes with respect to the weights and biases we can leverage the chain rule

\begin{align*}
\nabla_{\mathbf{W}_j} \mathcal{L}(\theta) &= \frac{d \mathcal{L}(\theta)}{d \mathbf{h}_j} \frac{d \mathbf{h}_j}{d \mathbf{z}_j}\frac{d \mathbf{z}_j}{d \mathbf{W}_j}\\[1ex]
\nabla_{\mathbf{b}_j} \mathcal{L}(\theta) &= \frac{d \mathcal{L}(\theta)}{d \mathbf{h}_j} \frac{d \mathbf{h}_j}{d \mathbf{z}_j}\frac{d \mathbf{z}_j}{d \mathbf{b}_j}
\end{align*}

As an example, let's focus on the gradients of the $d^{th}$ layer. Let $\mathbf{z}_d = \mathbf{W}_d^T \mathbf{h}_{d-1} + \mathbf{b}_d$, $f(\mathbf{x}) = \sigma(\mathbf{z}_d)$, and $\sigma^\prime(\mathbf{z}) = \sigma(\mathbf{z})(1 - \sigma(\mathbf{z}))$. Then based on the above configuration,

\begin{align*}
 \frac{d \mathcal{L}(\theta)}{d \sigma(\mathbf{z}_d)} \frac{d \sigma(\mathbf{z}_d) }{d\mathbf{z}_d} &= - \sum_{i=1}^N (y_i (1 - \sigma(\mathbf{z}_d)) - (1- y_i) \sigma(\mathbf{z}_d)) \quad \text{ and }\\[1ex]
\frac{d \mathbf{z}_d}{d \mathbf{W}_d} &= \mathbf{h}_{d-1}\\[2ex]
\therefore \quad \nabla_{\mathbf{W}_j} (\theta) &= - \sum_{i=1}^N (y_i - \sigma(\mathbf{z}_d)) \mathbf{h}_{d-1}
\end{align*}

Similarly, we can leverage the first two derivatives above to find  

\begin{align*}
\nabla_{\mathbf{b}_d} \mathcal{L}(\theta) &= \frac{d \mathcal{L}(\theta)}{d \sigma(\mathbf{z}_d)} \frac{d \sigma(\mathbf{z}_d) }{d\mathbf{z}_d} \frac{d \mathbf{z}_d}{d \mathbf{b}_d} \\[1ex]
&= - \sum_{i=1}^N (y_i - \sigma(\mathbf{z}_d)) *1
\end{align*}


#### Parameter Updates

After we've computed the gradients we can then update the weights and biases at iteration $t+1$ via the following,

\begin{align*}
\mathbf{W}_j^{t+1} &= \mathbf{W}_j^{t} - \alpha \nabla_{\mathbf{W}_j} \mathcal{L}(\theta)\\[1ex]
\mathbf{b}_j^{t+1}&=\mathbf{b}_j^{t} - \alpha \nabla_{\mathbf{b}_j} \mathcal{L}(\theta)
\end{align*}

where $\alpha$ is the learning rate.

<br><br><br>

In this example, `mb_sgd` is a dictionary that stores the weigth and bias gradients while the `inter_grads` stores the intermediate chain rule gradients with respect to $d \mathbf{z}_j$  and $d \mathbf{h}_j$

<br>


In [ ]:
# Backpropagation function
# Inputs: fwd_dat dictionary from fwd calcs, B = size of mini-batch, d = layers,
# param = initialized params data, X,Y data
def backprop(fwd_dat, param,  B, d,  X, Y):
    mb_sgd = {} # Final output dictionary w/ dw_j and db_j
    inter_grads = {} # Intermediate dict for dz_j and dh_j gradients
    
    for i in range(d, 0, -1):
        dw_str = "dw" + str(i) # gradient for w_j
        db_str = "db" + str(i) # gradient for b_j
        dz_str, dh_str = "dz" + str(i), "dh" + str(i)
        z_str, h_str = "z" + str(i), "h" + str(i)
        
        
        # End layer computation dw and db
        if i == d:
            # compute dz of final layer
            inter_grads[dz_str] = fwd_dat[h_str] - Y
            
            h_str_prev = "h"+ str(i -1)
            h_prev = fwd_dat[h_str_prev]
            
            # compute dw and db from final layer
            mb_sgd[dw_str] = (1./B) * np.matmul(inter_grads[dz_str], h_prev.T)
            mb_sgd[db_str] = (1./B) * np.sum(inter_grads[dz_str], axis=1, keepdims=True)
            
            
        # First layer dw and db computations
        elif i == 1:
            
            # Compute dh and dz from second layer
            w_str_fwd = "w" + str(i + 1)
            dz_str_fwd = "dz" + str(i + 1)
            
            inter_grads[dh_str] = np.matmul(param[w_str_fwd].T, inter_grads[dz_str_fwd])
            inter_grads[dz_str] = inter_grads[dh_str]*sigmoid(fwd_dat[z_str]) * (1 - sigmoid(fwd_dat[z_str]))
            
            # Compute dw and db for first layer
            mb_sgd[dw_str] = (1. / B)* np.matmul(inter_grads[dz_str], X.T)
            mb_sgd[db_str] = (1. / B)* np.sum(inter_grads[dz_str], axis = 1, keepdims=True)
    
        
        # Intermediate layer computations dw and db
        else:
            
            # Compute dz and dh from layers in front
            w_str_fwd = "w" + str(i + 1)
            dz_str_fwd = "dz" + str(i + 1)
            
            inter_grads[dh_str] = np.matmul(param[w_str_fwd].T, inter_grads[dz_str_fwd])
            inter_grads[dz_str] = inter_grads[dh_str]*sigmoid(fwd_dat[z_str]) * (1 - sigmoid(fwd_dat[z_str]))
            
            # Backprop towards layers behind to compute dw and db
            h_str_prev = "h"+ str(i -1)
            h_prev = fwd_dat[h_str_prev]
            
            # Compute dw and db
            mb_sgd[dw_str] = (1. / B) * np.matmul(inter_grads[dz_str], h_prev.T)
            mb_sgd[db_str] = (1. / B) * np.sum(inter_grads[dz_str], axis=1, keepdims=True)
        
      
    return mb_sgd
    
    

## Train Network

For this example I built a 3-layered Neural Network (input layer + 1 hidden layer + output layer) where each layer has 64 neurons (these can be changed by modifying `d`, and modifying the `n_h` parameter in the `initialize_params` function).

The network was trained over 40 `epochs`, with a mini `batch_size` of 500 for the backprop step, with a `learning_rate` of 1. 

The train and test loss is printed every 5 epochs (shown below). As we can see, both losses are decreasing as the number of epochs increases which indicates the network is learning.

In [82]:
# Use helper function to initialize weights and biases into dictionary
d = 3 # Includes first layer and final layer (i.e. d = 3 is a 2 hidden layer NN)
start_params = initialize_params(layers= d)


epochs = 40 # Set number of epochs
batch_size = 500 # Set batch-size
learning_rate = 1. # set learning rate
np.random.seed(10)

# Since X_train, y_train are in order, shuffle to avoid any potential bias
perm = np.random.permutation(X_train.shape[1])
X_train_perm = X_train[:, perm]
y_train_perm = y_train_new[:, perm]

# For each epoch
for i in range(epochs):
     
    # Loop through all data, split into batch_size defined mini-batches
    # to estimate gradients
    for j in range(X_train.shape[1] / batch_size):

        # Get mini batch
        start = j * batch_size # get start idx
        end = min(start + batch_size, X_train.shape[1] - 1) # get end of batch
        X_mb = X_train_perm[:, start:end]
        y_mb = y_train_perm[:, start:end]

        # Calculate current fwd data and gradients
        fwd_dat = forward(start_params, X_mb, d)
        mb_sgd = backprop(fwd_dat, start_params, batch_size, d, X_mb, y_mb )

        # Based on gradients from backprop, update weights and biases
        for k in range(1, d +1):
            w_str, b_str = "w" + str(k), "b" + str(k)
            dw_str, db_str = "dw" + str(k), "db" + str(k)
            
            # Update w_j and b_j
            start_params[w_str] = start_params[w_str] - learning_rate * mb_sgd[dw_str]
            start_params[b_str] = start_params[b_str] - learning_rate * mb_sgd[db_str]


    # Calculate training set accuracy
    fwd_dat = forward(start_params, X_train, d)
    y_pred = fwd_dat["h" + str(d)]
    train_loss = cross_entropy(y_train_new, y_pred)

    # Calculate training set accuracy
    fwd_dat = forward(start_params, X_test, d)
    y_pred = fwd_dat["h" + str(d)]
    test_loss = cross_entropy(y_test_new, y_pred)
    
    # Print train and test losses every 5 epochs
    if i % 5 == 0:
        print("Epoch {}: test loss = {}, train loss = {}".format(i + 1, test_loss, train_loss))


Epoch 1: test loss = 0.818925043526, train loss = 0.846208574588
Epoch 6: test loss = 0.348094883373, train loss = 0.34890110104
Epoch 11: test loss = 0.252851299231, train loss = 0.238522267701
Epoch 16: test loss = 0.20601395031, train loss = 0.180581588354
Epoch 21: test loss = 0.178653547647, train loss = 0.143960062014
Epoch 26: test loss = 0.161516956541, train loss = 0.117924419292
Epoch 31: test loss = 0.150527946295, train loss = 0.0980818395856
Epoch 36: test loss = 0.14358941241, train loss = 0.0824589133178


## Generate Predictions

After the network is trained we compute the forwardfeed step with the most recent weights and biases, and extract the $arg \; max$ $f(\mathbf{x})$ of the softmax activation (i.e. the output layer for a single input $\mathbf{x}$). 

This will define the classification per each input $\mathbf{x}$, and thus the classifications for all inputs in $\mathbf{X}_{train}$ and $\mathbf{X}_{test}$. `y_pred_train` and `y_pred_test` are $n$ and $m$ length vectors, respectively, of in-sample and out-of-sample predicted labels.

In [83]:
# Training data predictions
fwd_dat_train = forward(start_params, X_train, d)
y_pred_train = np.argmax(fwd_dat_train["h" + str(d)], axis=0)

# Test data predictions
fwd_dat_test = forward(start_params, X_test, d)
y_pred_test = np.argmax(fwd_dat_test["h" + str(d)], axis=0)

## Network Performance

We then assess the networks in-sample and out-of-sample performance.

### Overall Accuracy

First, we compute the overall accuracy and see that in-sample performance is $99\%$ while the out-of-sample performance is $>97\%$.

In [84]:
train_acc = accuracy_score(y_train[0], y_pred_train) # Training accuracy
test_acc = accuracy_score(y_test[0], y_pred_test) # Test accuracy

# Report training/test accuracy
pd.DataFrame(np.array([train_acc, test_acc]).reshape(1,2),
            columns = ["Training Acc", "Test Acc"])


,Training Acc,Test Acc
0,0.990183,0.9746


### Training data classification report
Looking that the f1-score, we can see that the the network provides good in-sample classifiction across all digits.

We look at the f1-scre since given ti provides an overall performance metric for the algorithm that takes into account precision and recall.

In [85]:
pd.DataFrame(data = classification_report(y_train[0], y_pred_train, output_dict=True))

,0,1,2,3,4,5,6,7,8,9,macro avg,micro avg,weighted avg
f1-score,0.993413,0.993100,0.989951,0.988468,0.988464,0.989708,0.993334,0.989641,0.989315,0.986076,0.990147,0.990183,0.990184
precision,0.993749,0.993469,0.987801,0.991306,0.986862,0.994966,0.992078,0.988067,0.989569,0.984095,0.990196,0.990183,0.990194
recall,0.993078,0.992732,0.992111,0.985647,0.990072,0.984505,0.994593,0.991221,0.989062,0.988065,0.990109,0.990183,0.990183
support,5923.000000,6742.000000,5958.000000,6131.000000,5842.000000,5421.000000,5918.000000,6265.000000,5851.000000,5949.000000,60000.000000,60000.000000,60000.000000


### Test data classification report
Now, looking at the out of sample performance we can see that the network accurately classifies 0s the best while it classifies 9s and 5s not as well.

In [86]:
pd.DataFrame(data = classification_report(y_test[0], y_pred_test, output_dict=True))

,0,1,2,3,4,5,6,7,8,9,macro avg,micro avg,weighted avg
f1-score,0.983240,0.986784,0.975255,0.971513,0.973083,0.968768,0.973877,0.972317,0.971635,0.967359,0.974383,0.9746,0.974593
precision,0.978766,0.986784,0.976676,0.963938,0.970618,0.981588,0.974895,0.970902,0.976166,0.965449,0.974578,0.9746,0.974638
recall,0.987755,0.986784,0.973837,0.979208,0.975560,0.956278,0.972860,0.973735,0.967146,0.969277,0.974244,0.9746,0.974600
support,980.000000,1135.000000,1032.000000,1010.000000,982.000000,892.000000,958.000000,1028.000000,974.000000,1009.000000,10000.000000,10000.0000,10000.000000


## References

1) [Jonathan Weisberg - Building a Neural Network from Scratch: Part 1](https://jonathanweisberg.org/post/A%20Neural%20Network%20from%20Scratch%20-%20Part%201/)

2) [Jonathan Weisberg - Building a Neural Network from Scratch: Part 2](https://jonathanweisberg.org/post/A%20Neural%20Network%20from%20Scratch%20-%20Part%202/)